### Progress-1

In [3]:
import time
from tqdm import tqdm

# 模擬一個長時間運行的任務
for i in tqdm(range(100), desc="Loading..."):
    time.sleep(0.1)  # 模擬任務

Loading...: 100%|██████████| 100/100 [00:10<00:00,  9.79it/s]


### Progress-2 (infiniti loading)

In [1]:
import itertools
import sys
import time

def always_loading(desc="Loading...", delay=0.1):
    spinner = itertools.cycle(['-', '\\', '|', '/'])
    while True:
        sys.stdout.write(f'\r{desc} {next(spinner)}')
        sys.stdout.flush()
        time.sleep(delay)

try:
    always_loading()
except KeyboardInterrupt:
    sys.stdout.write('\rDone!     \n')
    sys.stdout.flush()

Done!      -


### Progress-3 (countdown)

In [8]:
import time
from tqdm import tqdm
from datetime import datetime, timedelta

def countdown_to(target_time):
    now = datetime.now()
    target = datetime.strptime(target_time, "%Y-%m-%d %H:%M:%S")
    delta = (target - now).total_seconds()

    if delta <= 0:
        print(f"Target date is expired => {target_time}")
        return

    for i in tqdm(range(int(delta)), desc="AWaiting time for auto-apply execution", unit="s", bar_format="{desc}: {remaining}"):
        time.sleep(1)

# 設定目標時間
target_time = "2024-11-14 23:21:00"
countdown_to(target_time)

Target Date is expired => 2024-11-14 23:21:00


In [1]:
import time, configparser

from jupyterlab_server.config import load_config
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# %lsmagic =>列出所有jupyter notebook magic commands

In [2]:
import logging
def init_log():
    import logging

    # 設定日誌格式
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    
    # 建立日誌記錄器
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.INFO)
    
    # 建立文件處理器
    file_handler = logging.FileHandler('example.log')
    file_handler.setLevel(logging.INFO)
    file_handler.setFormatter(formatter)
    
    # 建立控制台處理器
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_handler.setFormatter(formatter)
    
    # 將處理器添加到日誌記錄器
    logger.addHandler(file_handler)
    logger.addHandler(console_handler)
    return logger

In [3]:

def load_config(config_file):
    config = configparser.ConfigParser()
    # config_path = %pwd
    config.read(config_file)
    # load leave application data
    default_config = {key: config.get("default", key) for key in config.options("default")}
    apply_data = {key: config.get("apply_data", key) for key in config.options("apply_data")}
    return default_config, apply_data



In [5]:
config_path = %pwd
config_path += "/src/config.ini"
logger = init_log()

default_config, apply_data = load_config(config_path)

submit_form_id = default_config["submit_form_id"]
submit_button_id = default_config["submit_button_id"]
base_url = default_config["base_url"]
payload = apply_data

url = base_url + "/viewform" + "?" + "&".join([f"{key}={value}" for key, value in payload.items()])

options = webdriver.ChromeOptions()
for option in default_config["browser_options"].split(","):
    options.add_argument(option)

with webdriver.Chrome(options) as driver:
    driver.get(url)
    form = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, submit_form_id))
    )
    button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, submit_button_id))
    )

    # button.click()

    # if EC.url_matches(url + "/formResponse"):
    #     params = "\n".join([f"{key}={value}" for key, value in payload.items()])
    #     logger.info(f"Send request successfully with params: \n {params}" )

    time.sleep(5)

### Get Chromedriver install path

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

def get_chromedriver_path():
    try:
        # 初始化 Chrome WebDriver
        service = Service()
        driver = webdriver.Chrome(service=service)
        # 取得 ChromeDriver 的執行檔路徑
        chromedriver_path = service.path
        driver.quit()
        return chromedriver_path
    except Exception as e:
        print(f"Error occurred: {e}")
        return None

chromedriver_path = get_chromedriver_path()
if chromedriver_path:
    print(f"ChromeDriver path: {chromedriver_path}")
else:
    print("Failed to get ChromeDriver path.")

ChromeDriver path: /Users/shihxuancheng/.cache/selenium/chromedriver/mac-arm64/131.0.6778.85/chromedriver


In [20]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

# service = Service('/Users/shihxuancheng/.cache/selenium/chromedriver/mac-arm64/131.0.6778.85/chromedriver')
# service.start()
options = Options()
driver = webdriver.Remote('http://127.0.0.1:57922',options=options)
# driver.get('http://www.google.com/');
time.sleep(5) # Let the user actually see something!
driver.quit()